In [5]:
#import pydicom
import os
import numpy as np
import pandas as pd
import cv2
from tqdm.notebook import tqdm

TRAIN_IMG_FOLDER_PATH = "/workspace/cv16/cv16-Wallpaper-defect_classification-/data/train/PNG"
TRAIN_CSV_PATH = "/workspace/cv16/cv16-Wallpaper-defect_classification-/data/train.csv"
root_dir = '/workspace/data/train'

#한글로 이미지 읽기 쓰기

def han_imread(ld_dir, color_mode = True) :
    img_array = np.fromfile(ld_dir, np.uint8)
    if color_mode == True :
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    else :
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    return img

def han_imwrite(sv_dir, image) :
    new_img_name = sv_dir
    extension = os.path.splitext(new_img_name)[1] # 이미지 확장자
    result, encoded_img = cv2.imencode(extension, image)
    if result:
        with open(new_img_name, mode='w+b') as f:
            encoded_img.tofile(f)

png_dir = TRAIN_IMG_FOLDER_PATH
png_384_dir = TRAIN_IMG_FOLDER_PATH + '_384'
os.makedirs(png_dir, exist_ok=True)
os.makedirs(png_384_dir, exist_ok=True) # 사이즈는 훈련할때 미리 원하는 사이즈로 있어야 빠름. 원하는 preprocessing이나 사이즈는 미리 넣어주세요

meta_dict_list = []
i = 0
for (root, dirs, files) in tqdm(os.walk(root_dir)): # 디렉토리 내의 모든 파일 순회
    for file_name in tqdm(files):
        if ('png' in file_name) or ('jpg' in file_name) or ('bmp' in file_name) : # 무슨 확장자가 있는지 몰라서 일단 썼습니다
            meta_dict = dict()
            png_img_name = os.path.basename(root) + '_' + file_name.split('.')[0] + '.png'
            meta_dict['id'] = i
            meta_dict['filename'] = png_img_name
            
            meta_dict['label'] = os.path.basename(root) # basename : 파일 이름만 출력한다
            meta_dict_list.append(meta_dict)
            
            img = han_imread(os.path.join(root, file_name))
            img384 = cv2.resize(img, dsize = (384, 384), interpolation = cv2.INTER_AREA)

            han_imwrite(os.path.join(png_dir, png_img_name), img)
            han_imwrite(os.path.join(png_384_dir, png_img_name), img384)
            
            i += 1

meta_train = pd.DataFrame(meta_dict_list)
meta_train.to_csv(TRAIN_CSV_PATH, index = False)


In [6]:
meta_tarin_k = pd.read_csv(TRAIN_CSV_PATH)
meta_tarin_k

,id,filename,label
0,0,피스_44.png,피스
1,1,피스_23.png,피스
2,2,피스_33.png,피스
3,3,피스_29.png,피스
4,4,피스_27.png,피스
...,...,...,...
3452,3452,걸레받이수정_180.png,걸레받이수정
3453,3453,걸레받이수정_111.png,걸레받이수정
3454,3454,걸레받이수정_4.png,걸레받이수정
3455,3455,걸레받이수정_81.png,걸레받이수정
